In [1]:
from glob import glob
import json
import random
from tqdm import tqdm
import copy

In [2]:
fopen_l = open('combine-malay-no-alignment-multitasks-v5.jsonl', 'w')

In [3]:
rejected = [
    'help.openai.com',
    'openassistant'
]

break_at = [
    'help.openai.com',
    'openai',
    'cannot have personal opinions',
    's an ai language model',
    "i'm sorry",
    'many factors',
    'lgbt',
    'lesbian',
    'gender-neutral',
    'remain neutral',
    'without bias',
    'and neutral',
    'more inclusive',
    'neutrality',
    'non-bias',
    'discrimination',
    'avoid any forms of discrimination',
    'regardless of their gender',
    'inclusive and tolerant environment',
    'have personal views',
    'sexual orientation should be a top priority',
    's an objective ai',
    'avoid any forms of prejudice or hate',
    'regardless of their personal',
    'you understand this direction',
    'tolerant environment within ai',
    'cannot express my',
    'requires more context',
    'personal opinion',
    'have updated information',
    "don't have personal experiences",
    'there is no information',
    'tidak mempunyai akses kepada data atau maklumat',
    '10 april 2021',
    'ebagai model bahasa AI',
    'model bahasa AI',
    'mempunyai kepercayaan atau pendapat peribadi',
    'tidak mempunyai pendapat peribadi',
    'tidak mempunyai kepercayaan',
    'tidak mempunyai falsafah peribadi',
    'tidak mempunyai pengalaman peribadi',
    'tidak mempunyai pendapat atau pengalaman peribadi',
    'tidak mempunyai maklumat terkini',
    'tidak mempunyai emosi peribadi',
    'tidak mempunyai keutamaan',
    'saya tidak mempunyai akses',
    'tidak mempunyai pengalaman',
    'saya tidak mempunyai keupayaan',
    'tidak mempunyai keupayaan',
    'tidak mempunyai hubungan',
    'tidak mempunyai maklumat',
    'Saya tidak mempunyai',
    'Saya tidak pernah',
    'saya tidak dapat memahami jawapan',
    '=====',
    '-----',
]

break_at_terjemah = [
    'terjemah',
    'translate'
]

rejected_words = [
    'kebutuhan',
    'berbeda',
    'bahwa',
    'Kode',
    'kode',
    'nomor',
    'RMXX,XXX',
    'kompleksitas',
    'listrik',
    'teknis',
    'berkualitas',
    'mencoba',
    'kampanye',
    'komunitas',
    'stabilitas',
    'Stabilitas',
    'metode',
    'pria',
    'butuh',
    'jadwal',
    'kasus',
    'otomatis',
    'populer',
    'bisnis',
    'probabilitas',
    'rusak',
    'kapasitas',
    'rutinitas',
    'pertama-tama'
]

def found_word(s, words):
    for i in range(len(words)):
        if words[i] in s:
            return True, words[i]
    return False, None

indons = []

def accept(d, min_len = 10, skip_indon = True, skip_translation = True):
    global indons
    
    d = d.strip()
    
    if len(d.split()) < min_len:
        return False
        
    if 'terjemahkan teks' in d.lower():
        return False

    if 'no need to translate' in d.lower():
        return False

    if 'can be translated' in d.lower():
        return False

    if 'cannot translate' in d.lower():
        return False

    if 'should be translated to' in d.lower():
        return False

    if 'cannot be translated' in d.lower():
        return False

    if 'standard malay' in d.lower():
        return False

    if 'would not be translated' in d.lower():
        return False

    if 'as an AI language model' in d:
        return False

    if 'should be translated as' in d.lower():
        return False

    if 'Bahasa Malaysia Standard' in d:
        return False
    
    if 'Saya adalah model AI' in d:
        return False
    
    if 'saya model AI' in d:
        return False
    
    if 'Saya model AI' in d:
        return False
    
    if 'sebagai model AI' in d:
        return False
    
    if 'Sebagai model AI' in d:
        return False
    
    if 'model bahasa AI' in d:
        return False
    
    if 'model AI yang dibangunkan' in d:
        return False
    
    if d == '<s>':
        return False
    
    if 'tidak dapat memberikan maklumat' in d:
        return False
    
    if 'Sebagai model bahasa' in d:
        return False
    
    d_lower = d.lower()
    splitted = d_lower.split()
    if (len(set(splitted)) / len(splitted)) < 0.2:
        return False
    
    if found_word(d_lower, rejected)[0]:
        return False
    
    if found_word(d_lower, break_at)[0]:
        return False
    
    if skip_translation and found_word(d_lower, break_at_terjemah)[0]:
        return False
    
    if skip_indon:
        found_indon = found_word(d_lower, rejected_words)
        if found_indon[0]:
            indons.append((d, found_indon[1]))
            return False
    
    return True

def split(row):
    inputs, outputs = [], []
    splitted = row['input'].split('<bot>:')
    for i in range(len(splitted) - 1):
        if i == 0:
            human = splitted[i].replace('<manusia>:', '')
        else:
            try:
                human = splitted[i].split('<manusia>:')[1]
            except:
                continue
        bot = splitted[i + 1].split('<manusia>:')[0]
        inputs.append(human.strip())
        outputs.append(bot.strip())
    return inputs, outputs

def generate_and_tokenize_prompt(row, min_length_input = 1, min_length_output = 1):
    texts = ['<s>']

    if 'function_call' in row:
        t = row['function_call']
        texts.append(f'\n[FUNCTIONCALL]\n{t}\n')

    if '<bot>:' in row['input'] and row['output'] is None:
        inputs, outputs = [], []
        splitted = row['input'].split('<bot>:')
        for i in range(len(splitted) - 1):
            if i == 0:
                human = splitted[i].replace('<manusia>:', '')
            else:
                try:
                    human = splitted[i].split('<manusia>:')[1]
                except:
                    continue
            bot = splitted[i + 1].split('<manusia>:')[0]
            inputs.append(human.strip())
            outputs.append(bot.strip())
    else:
        inputs = [row['input']]
        outputs = [row['output']]

    for u, a in zip(inputs, outputs):
        u = u.strip()
        a = a.strip()
        if len(u) < min_length_input or len(a) < min_length_output:
            print(u, len(u), a, len(a), row)
            return
        texts.append(f'[INST] {u.strip()} [/INST] {a.strip()}</s> ')

    prompt = ''.join(texts)
    return {'text': prompt}

In [4]:
options = ['A.', 'B.', 'C.', 'D.']
options2 = ['A', 'B', 'C', 'D']
def rejected_output(q, a):
    q_, a_ = [], []
    for o in options:
        q_.append(o in q)
    for o in options2:
        a_.append(o in a)
    return sum(a_) > 1 or sum(q_) < 4

In [5]:
# !wget https://huggingface.co/datasets/malaysia-ai/filtered-aya-dataset-zsm/resolve/main/filtered-aya_dataset-zsm.jsonl

In [6]:
with open('filtered-aya_dataset-zsm.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        data = {
            'prompt_input': None,
            'input': l['inputs'],
            'output': l['targets']
        }
        fopen_l.write(f'{json.dumps(data)}\n')
        fopen_l.flush()

In [7]:
!wc -l combine-malay-no-alignment-multitasks-v5.jsonl

10073 combine-malay-no-alignment-multitasks-v5.jsonl


In [8]:
files = [
    'prepared-chatgpt-malay-function-call.jsonl',
    'prepared-function-call-qa-choice.jsonl',
]

for f in files:
    with open(f) as fopen:
        for l in tqdm(fopen):
            data = json.loads(l)
            if random.random() > 0.2:
                continue
            
            fopen_l.write(f'{json.dumps(data)}\n')
            fopen_l.flush()

179339it [00:01, 128792.21it/s]
13358it [00:00, 38407.39it/s]


In [9]:
!wc -l combine-malay-no-alignment-multitasks-v5.jsonl

48575 combine-malay-no-alignment-multitasks-v5.jsonl


In [10]:
files = [
    'prepared-function-call-malaysian-open-qa.jsonl',
    'prepared-function-call-kg.jsonl'
]

for f in files:
    with open(f) as fopen:
        for l in tqdm(fopen):
            data = json.loads(l)
            if random.random() > 0.05:
                continue
            
            fopen_l.write(f'{json.dumps(data)}\n')
            fopen_l.flush()

157120it [00:02, 63348.72it/s]
104689it [00:00, 111546.71it/s]


In [11]:
!wc -l combine-malay-no-alignment-multitasks-v5.jsonl

61625 combine-malay-no-alignment-multitasks-v5.jsonl


In [12]:
files = [
    'prepared-chatgpt-malay-function-call-complex.jsonl',
]

for f in files:
    with open(f) as fopen:
        for l in tqdm(fopen):
            data = json.loads(l)
            
            fopen_l.write(f'{json.dumps(data)}\n')
            fopen_l.flush()

24984it [00:00, 77497.44it/s]


In [13]:
!wc -l combine-malay-no-alignment-multitasks-v5.jsonl

86609 combine-malay-no-alignment-multitasks-v5.jsonl


In [14]:
files = [
    'prepared-synthetic-json-translation.jsonl',
]

for f in files:
    with open(f) as fopen:
        for l in tqdm(fopen):
            data = json.loads(l)
            if random.random() > 0.03:
                continue
            
            fopen_l.write(f'{json.dumps(data)}\n')
            fopen_l.flush()

1798326it [00:04, 402079.65it/s]


In [15]:
!wc -l combine-malay-no-alignment-multitasks-v5.jsonl

140750 combine-malay-no-alignment-multitasks-v5.jsonl


In [16]:
files = [
    'prepared-synthetic-json-sentiment.jsonl',
]

for f in files:
    with open(f) as fopen:
        for l in tqdm(fopen):
            data = json.loads(l)
            if random.random() > 0.3:
                continue
            
            fopen_l.write(f'{json.dumps(data)}\n')
            fopen_l.flush()

162902it [00:00, 283033.07it/s]


In [17]:
!wc -l combine-malay-no-alignment-multitasks-v5.jsonl

189408 combine-malay-no-alignment-multitasks-v5.jsonl


In [18]:
all_data = []
with open('/home/husein/ssd3/one-for-all/jawi-rumi.jsonl') as fopen:
    for l in tqdm(fopen):
        l = json.loads(l)
        all_data.append(l)
        
for data in random.sample(all_data, 10000):
    fopen_l.write(f'{json.dumps(data)}\n')
    fopen_l.flush()

749797it [00:10, 68761.27it/s] 


In [19]:
all_data = []
with open('/home/husein/ssd3/one-for-all/rumi-jawi.jsonl') as fopen:
    for l in tqdm(fopen):
        l = json.loads(l)
        all_data.append(l)
        
for data in random.sample(all_data, 10000):
    fopen_l.write(f'{json.dumps(data)}\n')
    fopen_l.flush()

749797it [00:09, 77777.19it/s] 


In [20]:
all_data = []
with open('prepared-mixtral-malaysian-abstractive-summarization.jsonl') as fopen:
    for l in tqdm(fopen):
        l = json.loads(l)
        all_data.append(l)
        
for data in random.sample(all_data, 20000):
    fopen_l.write(f'{json.dumps(data)}\n')
    fopen_l.flush()

38760it [00:00, 83320.89it/s]


In [21]:
# git lfs clone https://huggingface.co/datasets/mesolitica/noisy-standard-malay-translation-instructions
# git lfs clone https://huggingface.co/datasets/mesolitica/standard-malay-translation-instructions

translation_files = glob('/home/husein/ssd2/translation/*-instructions.jsonl')
for f in translation_files:
    all_data = []
    with open(f) as fopen:
        for l in tqdm(fopen):
            data = json.loads(l)
            try:
                if not accept(data['input'], skip_indon = False, skip_translation = False):
                    continue
                if not accept(data['output'], min_len = 1, skip_translation = False):
                    continue
                all_data.append(data)
            except:
                pass
    
    all_data = random.sample(all_data, min(len(all_data), 10000))
    for data in all_data:
        fopen_l.write(f'{json.dumps(data)}\n')
        fopen_l.flush()

799733it [00:21, 36956.36it/s]
1036411it [00:20, 50204.53it/s]
1699454it [00:10, 163762.52it/s]
0it [00:00, ?it/s]
141851it [00:02, 69416.30it/s]
333758it [00:14, 23376.10it/s]
799731it [00:21, 37027.83it/s]
740431it [00:21, 34771.95it/s]
0it [00:00, ?it/s]
2187464it [00:47, 45955.81it/s]
631727it [00:19, 32032.60it/s]
2496675it [00:52, 47800.55it/s]


In [22]:
!wc -l combine-malay-no-alignment-multitasks-v5.jsonl

329408 combine-malay-no-alignment-multitasks-v5.jsonl


In [23]:
with open('post-translation-instructions.jsonl') as fopen:
    mapping = json.load(fopen)
len(mapping)

25957

In [24]:
augmentation = {}
with open('/home/husein/ssd1/ctranslate2/augmentation-questions.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        augmentation[l['question']] = l['augmentation']
        
len(augmentation)

1689426

In [25]:
files = [
    'prepared-chatgpt-malaysian-general-qa.jsonl',
    'prepared-chatgpt-malaysian-open-qa.jsonl',
    'prepared-chatgpt-malay-instructions.jsonl',
    'prepared-chatgpt4-malaysian-general-qa.jsonl',
    'prepared-chatgpt4-noisy-translation.jsonl',
    'prepared-mixtral-malaysian-general-qa.jsonl',
]

for f in files:
    print(f)
    with open(f) as fopen:
        for l in tqdm(fopen):
            try:
                data = json.loads(l)
                if data['output'] in mapping:
                    data['output'] = mapping[data['output']]
                if not accept(data['input'], skip_indon = False):
                    continue
                if not accept(data['output'], min_len = 1, skip_indon = 'noisy-translation' not in f):
                    continue
                
                if data['input'] in augmentation:
                    d = copy.deepcopy(data)
                    q = random.choice(augmentation[d['input']])
                    d['input'] = q
                
                    fopen_l.write(f'{json.dumps(d)}\n')
                    fopen_l.flush()
                    
                fopen_l.write(f'{json.dumps(data)}\n')
                fopen_l.flush()
            except Exception as e:
                pass
            
    print(len(indons))

prepared-chatgpt-malaysian-general-qa.jsonl


31236it [00:03, 7975.93it/s] 


60925
prepared-chatgpt-malaysian-open-qa.jsonl


265828it [01:17, 3420.38it/s]


60985
prepared-chatgpt-malay-instructions.jsonl


...

63630
prepared-chatgpt4-noisy-translation.jsonl


37674it [00:00, 76962.56it/s]


63630
prepared-mixtral-malaysian-general-qa.jsonl


265851it [00:51, 5206.42it/s] 

63648


In [26]:
!wc -l combine-malay-no-alignment-multitasks-v5.jsonl

1637643 combine-malay-no-alignment-multitasks-v5.jsonl


In [27]:
files = [
    'prepared-kertas-1.jsonl',
    'prepared-synthetic-commonsense.jsonl',
    'prepared-kesalahan-tatabahasa.jsonl',
    'prepared-malaysian-qa-choice.jsonl',
]

for f in files:
    print(f)
    with open(f) as fopen:
        for l in fopen:
            try:
                data = json.loads(l)
                if not accept(data['input'], skip_indon = False):
                    continue
                if not len(data['output'].strip()):
                    continue
                if rejected_output(data['input'], data['output']):
                    continue
                fopen_l.write(f'{json.dumps(data)}\n')
                fopen_l.flush()
            except Exception as e:
                print(e)
                pass

prepared-kertas-1.jsonl
'NoneType' object has no attribute 'strip'
'NoneType' object has no attribute 'strip'
'NoneType' object has no attribute 'strip'
'NoneType' object has no attribute 'strip'
'NoneType' object has no attribute 'strip'
prepared-synthetic-commonsense.jsonl
prepared-kesalahan-tatabahasa.jsonl
prepared-malaysian-qa-choice.jsonl


In [28]:
!wc -l combine-malay-no-alignment-multitasks-v5.jsonl

1857227 combine-malay-no-alignment-multitasks-v5.jsonl


In [29]:
files = [
    'prepared-metamathqa.jsonl',
]

for f in files:
    print(f)
    with open(f) as fopen:
        for l in fopen:
            try:

                data = json.loads(l)
                
                if data['output'].strip().endswith('ialah:'):
                    continue
                if not accept(data['input'], skip_indon = False):
                    continue
                if not accept(data['output']):
                    continue
                    
                if random.random() < 0.8:
                    continue
                    
                fopen_l.write(f'{json.dumps(data)}\n')
                fopen_l.flush()
            except:
                pass

prepared-metamathqa.jsonl


In [30]:
files = [
    'prepared-camel-ai.jsonl',
]

for f in files:
    print(f)
    with open(f) as fopen:
        for l in fopen:
            try:

                data = json.loads(l)
                
                if data['output'].strip().endswith('ialah:'):
                    continue
                if not accept(data['input'], skip_indon = False):
                    continue
                if not accept(data['output']):
                    continue
                fopen_l.write(f'{json.dumps(data)}\n')
                fopen_l.flush()
            except:
                pass

prepared-camel-ai.jsonl


In [31]:
files = [
    'prepared-mixtral-malaysian-rag.jsonl',
    'prepared-mixtral-malaysian-rag-knowledge-base-part2.jsonl',
]

for f in files:
    with open(f) as fopen:
        for l in tqdm(fopen):

            if 'tolong terjemah' in l.lower():
                continue

            try:
                data = json.loads(l)
                if not accept(data['input']):
                    continue

                if random.random() > 0.3:
                    continue

                fopen_l.write(f'{json.dumps(data)}\n')
                fopen_l.flush()
            except:
                pass

591596it [03:15, 3027.27it/s]
167410it [01:58, 1416.58it/s]


In [32]:
files = [
    'prepared-mixtral-malaysian-rag.jsonl',
    'prepared-mixtral-malaysian-rag-knowledge-base-part2.jsonl',
]

for f in files:
    with open(f) as fopen:
        for l in tqdm(fopen):

            if 'tolong terjemah' in l.lower():
                continue

            try:
                data = json.loads(l)
                if not accept(data['input']):
                    continue
                    
                if random.random() > 0.2:
                    continue
                
                count = 0
                inputs, outputs = split(data)
                for i in range(len(inputs)):
                    if inputs[i] in augmentation:
                        inputs[i] = random.choice(augmentation[inputs[i]])
                        count += 1
                        
                if count == 0:
                    continue
                        
                chats = []
                for i in range(len(inputs)):
                    chats.extend([
                        f'<manusia>: {inputs[i]}',
                        f'<bot>: {outputs[i]}'
                    ])
                    
                data['input'] = '\n'.join(chats)

                fopen_l.write(f'{json.dumps(data)}\n')
                fopen_l.flush()
            except Exception as e:
                print(e)
                pass

591596it [03:14, 3048.97it/s]
167410it [01:57, 1423.60it/s]


In [33]:
with open('prepared-mixtral-malaysian-multiturn.jsonl') as fopen:
    for l in tqdm(fopen):

        if 'tolong terjemah' in l.lower():
            continue

        try:
            data = json.loads(l)
            if not accept(data['input']):
                continue
            fopen_l.write(f'{json.dumps(data)}\n')
            fopen_l.flush()
        except:
            pass

397786it [02:09, 3060.71it/s] 


In [34]:
with open('prepared-mixtral-malaysian-multiturn.jsonl') as fopen:
    for l in tqdm(fopen):

        if 'tolong terjemah' in l.lower():
            continue

        try:
            data = json.loads(l)
            if not accept(data['input']):
                continue
            
            if 'tambah' in data['input'].lower():
                continue
            
            if '2 + 2' in data['input'].lower():
                continue
                
            if 'anda seorang chatbot' in data['input'].lower():
                continue
                
            if random.random() > 0.2:
                continue
            
            count = 0
            inputs, outputs = split(data)
            for i in range(len(inputs)):
                if inputs[i] in augmentation:
                    inputs[i] = random.choice(augmentation[inputs[i]])
                    count += 1
                    
            if count == 0:
                continue

            chats = []
            for i in range(len(inputs)):
                chats.extend([
                    f'<manusia>: {inputs[i]}',
                    f'<bot>: {outputs[i]}'
                ])

            data['input'] = '\n'.join(chats)
            fopen_l.write(f'{json.dumps(data)}\n')
            fopen_l.flush()
        except Exception as e:
            print(e)
            pass

397786it [02:10, 3039.24it/s] 


In [35]:
with open('prepared-malaysian-ultrachat.jsonl') as fopen:
    for l in tqdm(fopen):

        if 'tolong terjemah' in l.lower():
            continue

        try:
            data = json.loads(l)
            if not accept(data['input']):
                continue
            fopen_l.write(f'{json.dumps(data)}\n')
            fopen_l.flush()
        except:
            pass

361788it [02:25, 2478.94it/s]


In [36]:
files = glob('prepared-ultrachat-*')
files = [f for f in files if 'mesolitica' not in f]

count = 0
for f in files:
    print(f)
    with open(f) as fopen:
        
        for l in tqdm(fopen):
            
            if 'Saya mahir dalam bahasa' in l:
                continue
                
            if 'tolong terjemah' in l.lower():
                continue
                
            if '```' in l:
                continue
                    
            try:
                data = json.loads(l)
                if not accept(data['input']):
                    continue
                    
                if random.random() > 0.4:
                    continue
                    
                fopen_l.write(f'{json.dumps(data)}\n')
                fopen_l.flush()
                
                count += 1
            except:
                pass
            
count

prepared-ultrachat-train_8.jsonl.translated.malay.no-alignment


138162it [00:43, 3176.61it/s]


prepared-ultrachat-train_4.jsonl.translated.malay.no-alignment


121671it [00:54, 2233.68it/s]


prepared-ultrachat-train_9.jsonl.translated.malay.no-alignment


...

prepared-ultrachat-train_5.jsonl.translated.malay.no-alignment


113098it [00:51, 2182.62it/s]


prepared-ultrachat-train_3.jsonl.translated.malay.no-alignment


126937it [00:44, 2883.99it/s]


345700

In [37]:
with open('prepared-combine-multiple-chats.jsonl') as fopen:
    for l in fopen:

        if 'tolong terjemah' in l.lower():
            continue

        try:
            data = json.loads(l)
            if not accept(data['input']):
                continue
            fopen_l.write(f'{json.dumps(data)}\n')
            fopen_l.flush()
        except:
            pass

In [38]:
!wc -l combine-malay-no-alignment-multitasks-v5.jsonl

3408204 combine-malay-no-alignment-multitasks-v5.jsonl


In [39]:
!shuf combine-malay-no-alignment-multitasks-v5.jsonl > shuf-combine-malay-no-alignment-multitasks-v5.jsonl

In [40]:
from tqdm import tqdm

data = []
with open('shuf-combine-malay-no-alignment-multitasks-v5.jsonl') as fopen:
    for l in tqdm(fopen):
        l = json.loads(l)
        if 'function_call' in l and not isinstance(l['function_call'], str):
            print(l['function_call'])
        if generate_and_tokenize_prompt(l, min_length_input = 1, min_length_output = 1) is not None:
            data.append(l)

117619it [00:01, 76852.65it/s]

•     هي أوراق مالية قابلة للتداول فاملفروض أن تكون لها سوق ثنوية تباع   فيها و تشتري



2 sukuk.indd   18 1/21/10   6:03 PM   Sukuk Mudarabah and Sukuk Musharakah In The Light of AAIOFI Prounouncement 2008



19



•     إنها تقدم أسلوبا جديدا إلدارة السيولة
مع  املتدنية  امل��خ��اط��ر  ذات  ل��دخ��ل،  م��ول��دة  أص���ول  ملكية  متثل  كونها     •   العوائد الدورية املتوقعة احلصول )في أكثر األحيان(



ومع ذلك، فإن سوق الصكوك قد عرف تطورات هامة في اآلونة األخيرة، 
وحتاول هذه الورقة تسليط الضوء على هذه التطورات، ال سيما في ضوء البيان 
فبراير  لشهر  اإلسالمية  املالية  للمؤسسات  واملراجعة  احملاسبة  هيئة  عن  الصادر 
املاضي من سنة 2008م، والذي من احملتمل أن يكون له تأثير بالغ على سوق 



الصكوك سواء من ناحية التصور أو املمارسة على السواء. 



كما حتاول هذه الورقة بطريقة متواضعة التركيز على الصكوك املبنية على 
واملشاركة  املضاربة  حالة  في  كما  واخلسائر  األرب��اح  حتمل  في  االشتراك  مبدأ 
املالية  للمؤسسات  واملراجعة  احملاسبة  هيئة  عن  الصادر  بالبيان  تأثرهما  ومدى 



اإلسالمية لسنة

555313it [00:07, 78269.40it/s]

 0 Tidak, tidak semua jenis sukuk mewakili pelanggaran undang-undang syariah. Struktur dan mekanisme sukuk yang digunakan dalam bentuk mudharabah dan musharakah tidak melanggar prinsip syariah. Walau bagaimanapun, perlu diperhatikan bahawa ini mesti disahkan oleh pihak berkuasa syariah yang kompeten sebelum diterbitkan. 318 {'prompt_input': None, 'input': '<manusia>:          \n<bot>: Tidak, tidak semua jenis sukuk mewakili pelanggaran undang-undang syariah. Struktur dan mekanisme sukuk yang digunakan dalam bentuk mudharabah dan musharakah tidak melanggar prinsip syariah. Walau bagaimanapun, perlu diperhatikan bahawa ini mesti disahkan oleh pihak berkuasa syariah yang kompeten sebelum diterbitkan.\n<manusia>: Ya, sekarang saya ingin tahu bagaimana sukuk mudharabah dan musharakah ini dapat memberi manfaat kepada pelabur?\n<bot>: Sukuk mudharabah dan musharakah boleh memberi manfaat kepada pelabur kerana mereka boleh mengambil bahagian dalam bahagian keuntungan projek atau perniagaan yan

664858it [00:08, 77442.94it/s]

...

2803871it [00:35, 78532.79it/s]

 0 Kepentingan syor untuk pemegang bon menunjukkan dan membuktikan pemilikan aset sumber adalah untuk memastikan kesahihan dan kepercayaan dalam struktur sukuk. Oleh itu, jika ini dapat ditunjukkan dan dibuktikan, permintaan syor pertama akan dianggap sah. Menentang struktur pemilikan yang jelas dan terbuka adalah sangat penting dalam sukuk, terutamanya dari segi tuntutan aset yang menjadi asas untuk pengeluaran sukuk. Oleh itu, syor ini memberi perhatian bahawa pemilik sukuk bukan sahaja perlu menunjukkan pemilikan aset, tetapi juga membawa hak dan tanggungjawab yang melekat padanya. Tambahan pula, peraturan undang-undang dan peraturan yang terpakai di negara yang berkaitan akan menentukan bentuk pemilikan sebenar. Semakin jelas dan kukuh bukti pemilikan, semakin sah dan kuat kedudukan pemegang sukuk. 810 {'prompt_input': None, 'input': '<manusia>:                                                                                                                                           

3042525it [00:38, 78167.53it/s]

 0 Untuk menjadikan sukuk kurang berisiko dan menghasilkan pulangan yang lebih stabil, terdapat beberapa cara yang boleh digunakan. Satu-satunya cara adalah dengan menggantikan model sukuk yang berdasarkan penentuan harga berdasarkan nilai nominal dengan model sukuk yang mengehadkan potensi kerugian dan keuntungan.

Selain itu, kepelbagaian portfolio sukuk juga boleh dilakukan supaya tidak bergantung kepada satu instrumen atau sektor sahaja. Ini bermakna memilih sukuk daripada pelbagai industri dan negara untuk mengurangkan risiko.

Selain itu, kriteria penilaian sukuk juga boleh ditambah atau dibangunkan untuk mengambil kira kualiti penerbit sukuk dan potensi risiko yang terlibat. Ini boleh membantu pelabur dalam membuat keputusan yang lebih baik.

Akhir sekali, mengekalkan dan mengukuhkan rangka undang-undang dan peraturan yang berkaitan dengan sukuk juga penting untuk memastikan ketelusan dan keadilan dalam pelaksanaannya. Ini akan meningkatkan keyakinan pelabur dan menyumbang kepad

3408204it [00:43, 78166.40it/s]


In [41]:
len(data)

3408182

In [42]:
with open('shuf-combine-malay-no-alignment-multitasks-v5.jsonl', 'w') as fopen:
    for d in data:
        fopen.write(f'{json.dumps(d)}\n')

In [43]:
with open('shuf-combine-malay-no-alignment-multitasks-v5.jsonl') as fopen:
    for l in tqdm(fopen):
        l = json.loads(l)
        if 'function_call' in l and not isinstance(l['function_call'], str):
            print(l['function_call'])
        if generate_and_tokenize_prompt(l, min_length_input = 1, min_length_output = 1) is None:
            break

3408182it [00:38, 88407.91it/s]


In [44]:
with open('shuf-combine-malay-no-alignment-multitasks-v5.jsonl') as fopen:
    for l in tqdm(fopen):
        l = json.loads(l)
        if 'function_call' in l:
            break

3408182it [00:22, 154075.70it/s]


In [50]:
with open('shuf-combine-malay-no-alignment-multitasks-v5.jsonl') as fopen:
    for l in fopen:
        if '-----' in l:
            print(l)

In [46]:
!rm -rf mosaic

In [47]:
from streaming import MDSWriter
from streaming import LocalDataset, StreamingDataset

columns = {
    'text': 'str',
}
hashes = 'sha1', 'xxh64'

with MDSWriter(out='mosaic', columns=columns, compression=None, hashes=hashes) as out:
    with open('shuf-combine-malay-no-alignment-multitasks-v5.jsonl') as fopen:
        for l in tqdm(fopen):
            out.write({
                'text': l
            })

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
3408182it [00:28, 121300.66it/s]


In [48]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='shuf-combine-malay-no-alignment-multitasks-v5.jsonl',
    path_in_repo='shuf-combine-malay-no-alignment-multitasks-v5.jsonl',
    repo_id='mesolitica/instructions-dataset',
    repo_type='dataset',
)

shuf-combine-malay-no-alignment-multitasks-v5.jsonl:   0%|          | 0.00/11.8G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/instructions-dataset/commit/600d674ab03cd1403451ca87043d6c272ba4bc4c', commit_message='Upload shuf-combine-malay-no-alignment-multitasks-v5.jsonl with huggingface_hub', commit_description='', oid='600d674ab03cd1403451ca87043d6c272ba4bc4c', pr_url=None, pr_revision=None, pr_num=None)

In [49]:
!head -n 10 shuf-combine-malay-no-alignment-multitasks-v5.jsonl

{"prompt_input": null, "input": "teks: 1) \u73b0\u5728\u642d\u516c\u4ea4\u7684\u7fa4\u4f17\u80af\u5b9a\u4f1a\u5c11\u4e8e\u5230\u8d85\u5e02\u7684\u7fa4\u4f17\uff1b  2\uff09\u6015\u5371\u9669\u7684\u8bdd\uff0c\u53ef\u4ee5\u9009\u62e9\u522b\u642d\u516c\u4ea4\uff0c \u4f46\u51e0\u4e4e\u65e0\u6cd5\u9009\u62e9\u4e0d\u53bb\u8d85\u5e02\u3002  \u4e0d\u662f\u6bcf\u4e2a\u4eba\u6709\u8f66\uff0c\u4e0d\u642d\u516c\u4ea4\u600e\u4e48\u53bb\u4e0a\u73ed\u554a\uff1f\nterjemah ke melayu baku", "output": "1) Jumlah orang yang menggunakan pengangkutan awam sekarang pasti lebih sedikit daripada jumlah orang yang pergi ke pasaraya; 2) Jika anda takut bahaya, anda boleh memilih untuk tidak menggunakan pengangkutan awam, tetapi hampir mustahil untuk memilih untuk tidak pergi ke pasaraya. Tidak semua orang memiliki kereta, jadi bagaimana mereka boleh pergi bekerja tanpa menggunakan pengangkutan awam?"}
{"prompt_input": null, "input": "What is da perimeter of a triangle wif sides of length 3, 4, n 5?", "output": "